<a href="https://colab.research.google.com/github/mostafa-ja/Anomaly-detection/blob/main/autoencoder5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# we upload, in case that we cant parse logs(a lot of time and ram consumption)

# Mount Google Drive to save datasets
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#copy log file
!cp '/content/drive/MyDrive/zHDFS2/logs_train' '/content/'
!cp '/content/drive/MyDrive/zHDFS2/logs_ntest' '/content/'
!cp '/content/drive/MyDrive/zHDFS2/logs_atest' '/content/'
!cp '/content/drive/MyDrive/zHDFS2/log2index' '/content/'
!cp '/content/drive/MyDrive/zHDFS2/reduced_embeddings' '/content/'


In [118]:
import pandas as pd
import json
from sklearn.metrics.pairwise import cosine_similarity
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import os
import numpy as np


# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
with open('/content/reduced_embeddings', 'r') as json_file:
    embeddings = json.load(json_file)

In [5]:
len(embeddings[0])


10

In [6]:
# all unique templates in our train datasets

dataset = 'logs_train'
templates = set()

with open('/content/' + dataset, 'r') as f:
        for row in f:
          for temp in row.split():
            templates.add(temp)

print(sorted(templates))
print('nember of templates : ',len(templates))

['0', '1', '10', '11', '12', '13', '14', '15', '16', '2', '3', '4', '5', '6', '7', '8', '9']
nember of templates :  17


In [7]:
# all unique templates in our test datasets

datasets = ['logs_ntest','logs_atest']
templates = set()

for dataset in datasets:
  with open('/content/' + dataset, 'r') as f:
          for row in f:
            for temp in row.split():
              templates.add(temp)

print(sorted(templates))
print('nember of templates : ',len(templates))

['0', '1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '3', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '4', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '5', '50', '51', '6', '7', '8', '9']
nember of templates :  52


In [8]:

name = 'logs_train'
window_size = 10
num_sessions = 0
inputs = []
#outputs = []

with open('/content/' + name, 'r') as f:
        for row in f:
            num_sessions += 1
            line = [ embeddings[int(i)] for i in row.strip().split()]
            for i in range(len(line) - window_size):
                inputs.append(line[i:i + window_size])
                #outputs.append(line[i + window_size])

print('Number of sessions({}): {}'.format(name, num_sessions))
print('Number of seqs({}): {}'.format(name, len(inputs)))
#dataset = TensorDataset(torch.tensor(inputs, dtype=torch.float), torch.tensor(outputs))
dataset = TensorDataset(torch.tensor(inputs, dtype=torch.float))

Number of sessions(logs_train): 446578
Number of seqs(logs_train): 4704000


In [9]:
print(inputs[0])

[[-0.3891972303390503, 0.12810246646404266, -0.07853731513023376, -0.10766388475894928, 0.23173417150974274, -0.08228960633277893, 0.017653826624155045, -0.030952107161283493, 0.040218982845544815, 0.030139772221446037], [0.38630372285842896, -0.12673094868659973, -0.11596252024173737, -0.18444840610027313, 0.08815333247184753, -0.02099493145942688, 0.10797911882400513, -0.09289209544658661, -0.05879585072398186, 0.03421926125884056], [-0.3891972303390503, 0.12810246646404266, -0.07853731513023376, -0.10766388475894928, 0.23173417150974274, -0.08228960633277893, 0.017653826624155045, -0.030952107161283493, 0.040218982845544815, 0.030139772221446037], [-0.3891972303390503, 0.12810246646404266, -0.07853731513023376, -0.10766388475894928, 0.23173417150974274, -0.08228960633277893, 0.017653826624155045, -0.030952107161283493, 0.040218982845544815, 0.030139772221446037], [-0.21878856420516968, -0.2136467546224594, 0.04706411808729172, 0.20318950712680817, -0.24529194831848145, 0.02122925035

In [10]:
class Encoder(nn.Module):
    def __init__(self, input_dim, hid_dim, n_layers): #embedding dimensions
        super().__init__()

        self.hid_dim = hid_dim
        self.n_layers = n_layers

        self.rnn = nn.LSTM(input_dim, hid_dim, n_layers, bidirectional=True,batch_first=True)


    def forward(self, src):

        #src = [src len, batch size]


        outputs, (hidden, cell) = self.rnn(src)

        #outputs = [batch size,src len, hid dim * n directions]
        #hidden = [batch size,n layers * n directions,  hid dim]
        #cell = [batch size, n layers * n directions, hid dim]

        #outputs are always from the top hidden layer

        return outputs[:,-1:,:]  #outputs[:,-1:,:]=output of the end of blocks which is concatenated of two directions(we use -1: for keeping the dimensions)

class Decoder(nn.Module):
    def __init__(self, input_dim, hid_dim, n_layers):       #input _dim depends on encoder's output
        super().__init__()

        self.hid_dim = hid_dim
        self.n_layers = n_layers

        self.rnn = nn.LSTM(input_dim, hid_dim, n_layers, bidirectional=False,batch_first=True)

    def forward(self, src):

        #src = [src len, batch size]
        src = torch.tile(src, (1, 10, 1)) # give the output of encoder to each cell of decoder

        outputs, (hidden, cell) = self.rnn(src)

        #outputs = [batch size,src len, hid dim * n directions]
        #hidden = [batch size,n layers * n directions,  hid dim]
        #cell = [batch size, n layers * n directions, hid dim]

        #outputs are always from the top hidden layer

        return outputs

class Autoencoder(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder

    def forward(self, src):
        encoded = self.encoder(src)
        decoded = self.decoder(encoded)
        return decoded

In [18]:
encoder = Encoder(10,32,1)
decoder = Decoder(64,10,1)
model = Autoencoder(encoder, decoder).to(device)
model

Autoencoder(
  (encoder): Encoder(
    (rnn): LSTM(10, 32, batch_first=True, bidirectional=True)
  )
  (decoder): Decoder(
    (rnn): LSTM(64, 10, batch_first=True)
  )
)

In [20]:
input_data = torch.rand(1, 10, 10).to(device) #(batch,sequence_length,embedding)
output= model(input_data)
print(output.shape)

torch.Size([1, 10, 10])


In [21]:
# Calculate the number of parameters
num_params = sum(p.numel() for p in model.parameters())
print(f"Number of Parameters: {num_params}")

Number of Parameters: 14304


In [22]:
dataloader = DataLoader(dataset, batch_size=128)

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters())

In [23]:
len(dataloader) # regard to batch size

36750

In [24]:
for step, (seq) in enumerate(dataloader):
  print(seq[0].shape)
  break

torch.Size([128, 10, 10])


In [ ]:

num_epochs = 100
input_size = 10 # embedding size

# Train the model
start_time = time.time()
total_step = len(dataloader)
for epoch in range(num_epochs):  # Loop over the dataset multiple times
    train_loss = 0
    for step, (seq) in enumerate(dataloader):
        # Forward pass
        seq = seq[0].clone().detach().view(-1, window_size, input_size).to(device)
        output = model(seq)
        loss = criterion(output, seq.to(device))

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
    print('Epoch [{}/{}], train_loss: {:.4f}'.format(epoch + 1, num_epochs, train_loss / total_step))
elapsed_time = time.time() - start_time
print('elapsed_time: {:.3f}s'.format(elapsed_time))
print('Finished Training')


In [ ]:
seq[0].reshape(1,10,10)

In [55]:
input_data = seq[0].reshape(1,10,10)
output= model(input_data)
print(output.shape)

torch.Size([1, 10, 10])


In [56]:
a = input_data.cpu().detach().numpy()[0] - output.cpu().detach().numpy()[0]

In [57]:
np.sum(a[5])

0.073634475

In [58]:
torch.sum(seq[0][0]-seq[0][3])

tensor(-0.1362, device='cuda:0')

In [59]:
cosine_similarity(input_data.cpu().detach().numpy()[0],output.cpu().detach().numpy()[0])

array([[ 0.9610373 ,  0.02295969,  0.04059182, -0.00797994,  0.06608871,
        -0.11450423,  0.06367902, -0.538867  , -0.54200083, -0.5351264 ],
       [ 0.06265119,  0.99223757,  0.66928744,  0.9965995 ,  0.65959114,
         0.99361   ,  0.64131004, -0.45647153, -0.4406333 , -0.45387778],
       [ 0.20867422,  0.61992234,  0.99156976,  0.6591899 ,  0.9915739 ,
         0.6456291 ,  0.9953552 , -0.03214208, -0.0064817 , -0.00773428],
       [ 0.06265119,  0.99223757,  0.66928744,  0.9965995 ,  0.65959114,
         0.99361   ,  0.64131004, -0.45647153, -0.4406333 , -0.45387778],
       [ 0.20867422,  0.61992234,  0.99156976,  0.6591899 ,  0.9915739 ,
         0.6456291 ,  0.9953552 , -0.03214208, -0.0064817 , -0.00773428],
       [ 0.06265119,  0.99223757,  0.66928744,  0.9965995 ,  0.65959114,
         0.99361   ,  0.64131004, -0.45647153, -0.4406333 , -0.45387778],
       [ 0.20867422,  0.61992234,  0.99156976,  0.6591899 ,  0.9915739 ,
         0.6456291 ,  0.9953552 , -0.03214208

In [70]:
len(embeddings)

53

In [80]:
embeddings.append([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])

In [ ]:
embeddings.append([-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0])

In [81]:
embeddings[52]

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [82]:
embeddings[53]

[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]

In [83]:
def generate(name):
    # If you what to replicate the DeepLog paper results(Actually, I have a better result than DeepLog paper results),
    # you should use the 'list' not 'set' to obtain the full dataset, I use 'set' just for test and acceleration.
    hdfs = set()
    # hdfs = []
    with open('/content/' + name, 'r') as f:
        for row in f:
            line = [int(i) for i in row.strip().split()]
            line = line + [53] * (window_size + 1 - len(line)) #if the length of the line is less than windows size, it covers by 30 a template with zeros vector
            hdfs.add(tuple(line))
            # hdfs.append(tuple(line))
    print('Number of sessions({}): {}'.format(name, len(hdfs)))
    return hdfs

In [84]:
test_normal_loader = generate('logs_ntest')
test_abnormal_loader = generate('logs_atest')

Number of sessions(logs_ntest): 1091
Number of sessions(logs_atest): 4126


In [85]:
with torch.no_grad():
    for line in test_normal_loader:
        for i in range(len(line) - window_size):
            session = line[i:i + window_size]
            seq = [embeddings[temp] for temp in session]
            seq = torch.tensor(seq, dtype=torch.float).view(-1, window_size, input_size).to(device)
            output = model(seq)
            break
        break

In [89]:
session

(1, 0, 0, 0, 2, 3, 4, 2, 3, 2)

In [112]:
output[:,0,:].shape

torch.Size([1, 10])

In [94]:
np.array(embeddings).shape

(54, 10)

In [111]:
np.argmax(cosine_similarity(output.cpu().detach().numpy()[0,:,:],np.array(embeddings)) ,axis=1 )

array([1, 0, 0, 9, 2, 3, 4, 2, 3, 2])

In [117]:
cosine_similarity(output.cpu().detach().numpy()[:,3,:],np.array(embeddings))

array([[ 0.93364596, -0.4434544 ,  0.21914434,  0.19454281, -0.65147745,
         0.71474941,  0.53797707,  0.15221624, -0.06525082,  0.96791527,
        -0.5763621 , -0.31966809, -0.08581298,  0.3525665 , -0.51343656,
        -0.54429658, -0.57737701,  0.69773972,  0.21558633,  0.38540546,
         0.6757934 ,  0.67971468,  0.53628898,  0.30833949,  0.2453729 ,
         0.46184703,  0.71523791,  0.25231187,  0.29977316,  0.3604768 ,
         0.3592888 ,  0.63598188,  0.18424333,  0.61498045,  0.03822683,
         0.21031395,  0.38700118,  0.67300933,  0.42254237,  0.68351357,
        -0.30130725,  0.29635648, -0.40252998, -0.23298372,  0.7300521 ,
         0.22557606,  0.6841217 ,  0.68081174,  0.69443181, -0.40250046,
         0.18954596,  0.39670442,  0.        ,  0.30450481]])

In [121]:
matrix = cosine_similarity(seq.cpu().detach().numpy()[0],output.cpu().detach().numpy()[0])
np.diag(matrix)

array([0.94757855, 0.99033636, 0.9951027 , 0.93364596, 0.96575195,
       0.9642491 , 0.9754004 , 0.97802216, 0.9820594 , 0.9909105 ],
      dtype=float32)

In [124]:
np.min(np.diag(matrix))

0.9390317

In [127]:
with torch.no_grad():
    c=0
    w=0
    for line in test_normal_loader:
        for i in range(len(line) - window_size):
            session = line[i:i + window_size]
            seq = [embeddings[temp] for temp in session]
            seq = torch.tensor(seq, dtype=torch.float).view(-1, window_size, input_size).to(device)
            output = model(seq)
            matrix = cosine_similarity(seq.cpu().detach().numpy()[0],output.cpu().detach().numpy()[0])
            if np.min(np.diag(matrix))<0.9:
              w+=1
            else:
              c+=1

print(w)
print(c)

6396
2334


In [128]:
with torch.no_grad():
    c=0
    w=0
    for line in test_abnormal_loader:
        for i in range(len(line) - window_size):
            session = line[i:i + window_size]
            seq = [embeddings[temp] for temp in session]
            seq = torch.tensor(seq, dtype=torch.float).view(-1, window_size, input_size).to(device)
            output = model(seq)
            matrix = cosine_similarity(seq.cpu().detach().numpy()[0],output.cpu().detach().numpy()[0])
            if np.min(np.diag(matrix))<0.9:
              w+=1
            else:
              c+=1

print(w)
print(c)

70593
9773
